https://www.datacamp.com/community/tutorials/recommender-systems-python

In [2]:
import pandas as pd
dir = !kaggle datasets download -d sootersaalu/amazon-top-50-bestselling-books-2009-2019

In [3]:
# using the code below to unzip a file in case if you need to do so.
#import zipfile
#with zipfile.ZipFile("file.zip","r") as zip_ref:
#   zip_ref.extractall("targetdir")

In [5]:
df =pd.read_csv('dir/bestsellers with categories.csv')

In [6]:
df.head(10)

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
5,A Dance with Dragons (A Song of Ice and Fire),George R. R. Martin,4.4,12643,11,2011,Fiction
6,A Game of Thrones / A Clash of Kings / A Storm...,George R. R. Martin,4.7,19735,30,2014,Fiction
7,A Gentleman in Moscow: A Novel,Amor Towles,4.7,19699,15,2017,Fiction
8,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,4.7,5983,3,2018,Non Fiction
9,A Man Called Ove: A Novel,Fredrik Backman,4.6,23848,8,2016,Fiction


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


## Book Recommendation based on ratings only

In [8]:
# I am going to place the data that we need into a new variable since we do not need other columns for now
book_rec = df[['Name', 'Author', 'User Rating', 'Reviews']]
book_rec

,Name,Author,User Rating,Reviews
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350
1,11/22/63: A Novel,Stephen King,4.6,2052
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979
3,1984 (Signet Classics),George Orwell,4.7,21424
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665
...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331


In [9]:
# the amount of unique values in each column
book_rec.nunique()

Name           351
Author         248
User Rating     14
Reviews        346
dtype: int64

    As one Can see, we have 550 entries for column name, however 351 unique names, let's see if they are duplicates

In [10]:
# Counting how many times each name appears
book_rec['Name'].value_counts()

Publication Manual of the American Psychological Association, 6th Edition                                                    10
StrengthsFinder 2.0                                                                                                           9
Oh, the Places You'll Go!                                                                                                     8
The Very Hungry Caterpillar                                                                                                   7
The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change                                                  7
                                                                                                                             ..
The Handmaid's Tale                                                                                                           1
What Should Danny Do? (The Power to Choose Series)                                                      

In [11]:
book_rec[book_rec['Name'] == 'Publication Manual of the American Psychological Association, 6th Edition']

,Name,Author,User Rating,Reviews
271,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
272,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
273,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
274,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
275,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
276,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
277,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
278,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
279,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580
280,Publication Manual of the American Psychologic...,American Psychological Association,4.5,8580


    As we can see above those rows happen to be duplicates. In the next step, I am going to drop those duplicate rows

In [12]:
# dropping duplicate rows
book_rec.drop_duplicates(subset = ['Name','Author', 'User Rating', 'Reviews'], keep = 'first', inplace=True)

<ipython-input-12-9f69486e3d65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_rec.drop_duplicates(subset = ['Name','Author', 'User Rating', 'Reviews'], keep = 'first', inplace=True)


In [13]:
book_rec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352 entries, 0 to 546
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         352 non-null    object 
 1   Author       352 non-null    object 
 2   User Rating  352 non-null    float64
 3   Reviews      352 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 13.8+ KB


In [14]:
#Top 50 books based on ratings only
top_50 = book_rec.sort_values(by = 'User Rating', ascending=False).head(50).copy()
top_50

,Name,Author,User Rating,Reviews
82,Dog Man: A Tale of Two Kitties: From the Creat...,Dav Pilkey,4.9,4786
174,Humans of New York : Stories,Brandon Stanton,4.9,2812
146,"Goodnight, Goodnight Construction Site (Hardco...",Sherri Duskey Rinker,4.9,7038
289,Rush Revere and the First Patriots: Time-Trave...,Rush Limbaugh,4.9,3836
40,"Brown Bear, Brown Bear, What Do You See?",Bill Martin Jr.,4.9,14344
151,Hamilton: The Revolution,Lin-Manuel Miranda,4.9,5867
83,Dog Man: Brawl of the Wild: From the Creator o...,Dav Pilkey,4.9,7235
153,Harry Potter and the Chamber of Secrets: The I...,J.K. Rowling,4.9,19622
288,Rush Revere and the Brave Pilgrims: Time-Trave...,Rush Limbaugh,4.9,7150
155,Harry Potter and the Goblet of Fire: The Illus...,J. K. Rowling,4.9,7758


In [15]:
# Let's Look at the top 5

top_50.head()

,Name,Author,User Rating,Reviews
82,Dog Man: A Tale of Two Kitties: From the Creat...,Dav Pilkey,4.9,4786
174,Humans of New York : Stories,Brandon Stanton,4.9,2812
146,"Goodnight, Goodnight Construction Site (Hardco...",Sherri Duskey Rinker,4.9,7038
289,Rush Revere and the First Patriots: Time-Trave...,Rush Limbaugh,4.9,3836
40,"Brown Bear, Brown Bear, What Do You See?",Bill Martin Jr.,4.9,14344


    
The other question that we need to ask is a rating of 4.9 with 4786 reviews the same as 4.9 with 14344 reviews, of course not.

In [19]:
# Let's add a score, which is determined by a weighted factor Wf = r/Rtotal, r is the reviews for a book and Rtotal
# Score = Rating*(r/Rtotal)

Rtotal = book_rec['Reviews'].sum() 
Rtotal

3444414

In [27]:
m = book_rec['Reviews'].quantile(0.70)
m

10091.699999999999

In [30]:
C = book_rec['Reviews'].mean()
C

9785.267045454546

In [34]:
def weighted_rating(x, m=m, C=C):
    v = x['Reviews']
    R = x['User Rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [35]:
book_rec['score'] = book_rec.apply(weighted_rating, axis=1)

<ipython-input-35-f1084e79d788>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_rec['score'] = book_rec.apply(weighted_rating, axis=1)


In [36]:
book_rec.head()

,Name,Author,User Rating,Reviews,score
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,3601.508815
1,11/22/63: A Novel,Stephen King,4.6,2052,8132.564098
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,3399.958747
3,1984 (Signet Classics),George Orwell,4.7,21424,3136.553281
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,5563.351943


In [21]:
df['Genre'].unique()

array(['Non Fiction', 'Fiction'], dtype=object)

In [22]:
# There are 248 authors
authors = df['Author']
authors.head()

0                    JJ Smith
1                Stephen King
2          Jordan B. Peterson
3               George Orwell
4    National Geographic Kids
Name: Author, dtype: object

In [23]:
# top 50 authors with the most books
top_50 = df['Author'].value_counts().head(70)
top_50

Jeff Kinney                           12
Rick Riordan                          11
Gary Chapman                          11
Suzanne Collins                       11
American Psychological Association    10
                                      ..
Zhi Gang Sha                           2
William Davis                          2
Greg Mortenson                         2
Kristin Hannah                         2
Craig Smith                            2
Name: Author, Length: 70, dtype: int64

In [24]:
# Selecting authors with at least 5 written books
five_books_plus = top_50[top_50 > 4]
name_with_most_books = list(five_books_plus.index)
name_with_most_books

['Jeff Kinney',
 'Rick Riordan',
 'Gary Chapman',
 'Suzanne Collins',
 'American Psychological Association',
 'Dr. Seuss',
 'Gallup',
 'Rob Elliott',
 'Dav Pilkey',
 "Bill O'Reilly",
 'Stephenie Meyer',
 'Eric Carle',
 'Stephen R. Covey',
 'J.K. Rowling',
 'Sarah Young',
 'Harper Lee',
 'E L James',
 'The College Board',
 'Stieg Larsson',
 'Don Miguel Ruiz',
 'Giles Andreae',
 'Patrick Lencioni',
 'Roger Priddy',
 'Laura Hillenbrand',
 'John Grisham',
 'John Green',
 'R. J. Palacio',
 'Dale Carnegie']

In [25]:
top_author = df[df['User Rating'] > 4.6 ][['Author', 'User Rating']]
top_author.groupby('Author')[['User Rating', 'Author']].head()

,User Rating,Author
0,4.7,JJ Smith
2,4.7,Jordan B. Peterson
3,4.7,George Orwell
4,4.8,National Geographic Kids
6,4.7,George R. R. Martin
...,...,...
544,4.8,R. J. Palacio
546,4.7,Jen Sincero
547,4.7,Jen Sincero
548,4.7,Jen Sincero


In [26]:
df[df['Author'] == 'Rick Riordan']['User Rating'].mean()

4.7727272727272725

In [27]:
def auther_mean_rating(name):
    mean = df[df['Author'] == name]['User Rating'].mean()
    return mean

In [28]:
auther_mean_rating('Jeff Kinney')

4.8

In [29]:
def all_author_mean_rating(author_name):
    l = {}
    for i in author_name:
        avg = round(auther_mean_rating(i),2)
        l[i] = avg
    return l

dict = all_author_mean_rating(top_author['Author'])
        

In [30]:
{k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse = True)}

{'Bill Martin Jr.': 4.9,
 'Dav Pilkey': 4.9,
 'Sherri Duskey Rinker': 4.9,
 'Lin-Manuel Miranda': 4.9,
 'Sarah Young': 4.9,
 'Jill Twiss': 4.9,
 'Alice Schertle': 4.9,
 'Pete Souza': 4.9,
 'Rush Limbaugh': 4.9,
 'Nathan W. Pyle': 4.9,
 'Patrick Thorpe': 4.9,
 'Chip Gaines': 4.9,
 'Eric Carle': 4.9,
 'Emily Winfield Martin': 4.9,
 'Dr. Seuss': 4.88,
 'J. K. Rowling': 4.85,
 'Mark R. Levin': 4.85,
 'Brandon Stanton': 4.83,
 'National Geographic Kids': 4.8,
 'Ron Chernow': 4.8,
 'Michelle Obama': 4.8,
 'Atul Gawande': 4.8,
 'Jeff Kinney': 4.8,
 'David Goggins': 4.8,
 'Marjorie Sarnat': 4.8,
 'Brené Brown': 4.8,
 'Rod Campbell': 4.8,
 'Giles Andreae': 4.8,
 'Adam Mansbach': 4.8,
 'Margaret Wise Brown': 4.8,
 'Raina Telgemeier': 4.8,
 'Phil Robertson': 4.8,
 'Mitch Albom': 4.8,
 'Joanna Gaines': 4.8,
 'Hopscotch Girls': 4.8,
 'Ann Whitford Paul': 4.8,
 'Anthony Bourdain': 4.8,
 'Robert Munsch': 4.8,
 'Julia Child': 4.8,
 'Crispin Boyer': 4.8,
 'Amy Shields': 4.8,
 'Wizards RPG Team': 4.8,
